In [1]:
import numpy as np
import pickle
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *
import os
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn import *
from sklearn.metrics import *
sns.set()
import warnings
warnings.filterwarnings("ignore")
length = 277

2024-03-26 19:10:46.005990: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 19:10:46.038601: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 19:10:46.038626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 19:10:46.039311: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-26 19:10:46.043688: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18325623732082902229
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5803868160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18101141183903711527
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2024-03-26 19:10:47.094173: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:10:47.113562: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:10:47.113601: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:10:47.228306: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:10:47.228434: I external/local_xla/xla/stream_executor

In [3]:
train_values = pd.read_csv('./data/train_test/train_patients.csv').values
test_values = pd.read_csv('./data/train_test/test_patients.csv').values

print(train_values.shape)
print(test_values.shape)


# Separate the training and testing data, and one-hot encode Y:
X_train = train_values[:,:-2]
X_test = test_values[:,:-2]
y_train = train_values[:,-2]
y_test = test_values[:,-2]
X_test1 = X_test.reshape(-1, X_train.shape[1], 1)
y_test1=to_categorical(y_test)


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
X_train1 = X_train.reshape(-1, X_train.shape[1], 1)
y_train1=to_categorical(y_train)

X_val1 = X_val.reshape(-1, X_train.shape[1], 1)
y_val1=to_categorical(y_val)


(38751, 277)
(72794, 277)
X_train shape: (31000, 275)
X_val shape: (7751, 275)
y_train shape: (31000,)
y_val shape: (7751,)


In [4]:
from tensorflow.keras.layers import Activation, BatchNormalization

def mlpmodel():
    mlpmodel = Sequential()
    mlpmodel.add(Dense(275, input_shape=(X_train.shape[1],)))
    mlpmodel.add(Dense(50))
    mlpmodel.add(BatchNormalization())
    mlpmodel.add(Activation('relu'))
    mlpmodel.add(Dense(150))
    mlpmodel.add(BatchNormalization())
    mlpmodel.add(Activation('relu'))
    mlpmodel.add(Dense(900))
    mlpmodel.add(BatchNormalization())
    mlpmodel.add(Activation('relu'))
    mlpmodel.add(Dense(400))
    mlpmodel.add(BatchNormalization())
    mlpmodel.add(Activation('relu'))
    mlpmodel.add(Dense(7, activation='softmax'))
    
    return mlpmodel

mlp = mlpmodel()
mlp.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 275)               75900     
                                                                 
 dense_1 (Dense)             (None, 50)                13800     
                                                                 
 batch_normalization (Batch  (None, 50)                200       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 150)               7650      
                                                                 
 batch_normalization_1 (Bat  (None, 150)               600       
 chNormalization)                                       

2024-03-26 19:10:49.041354: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:10:49.041432: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:10:49.041450: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:10:49.041575: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:10:49.041591: I external/local_xla/xla/stream_executor

In [5]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)


In [6]:
class_weights

array([0.25901974, 2.27306057, 2.33468896, 2.37438725, 2.32942591,
       2.37766529])

In [8]:
import tensorflow.keras.backend as K

K.clear_session()


In [9]:
mlp.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min',min_delta=0.001, restore_best_weights=True)
history = mlp.fit(X_train, y_train1, validation_data=(X_val1, y_val1), epochs=50,callbacks=[early_stopping], verbose=1)


Epoch 1/50
969/969 [==============================] - 12s 11ms/step - loss: 0.1002 - accuracy: 0.9670 - val_loss: 0.0823 - val_accuracy: 0.9724
Epoch 2/50
969/969 [==============================] - 10s 11ms/step - loss: 0.0851 - accuracy: 0.9718 - val_loss: 0.0776 - val_accuracy: 0.9763
Epoch 3/50
969/969 [==============================] - 10s 11ms/step - loss: 0.0734 - accuracy: 0.9754 - val_loss: 0.0612 - val_accuracy: 0.9804
Epoch 4/50
969/969 [==============================] - 10s 11ms/step - loss: 0.0637 - accuracy: 0.9782 - val_loss: 0.0647 - val_accuracy: 0.9801
Epoch 5/50
969/969 [==============================] - 10s 11ms/step - loss: 0.0611 - accuracy: 0.9789 - val_loss: 0.0568 - val_accuracy: 0.9812
Epoch 6/50
969/969 [==============================] - 10s 10ms/step - loss: 0.0580 - accuracy: 0.9799 - val_loss: 0.0653 - val_accuracy: 0.9783
Epoch 7/50
969/969 [==============================] - 11s 11ms/step - loss: 0.0555 - accuracy: 0.9812 - val_loss: 0.0525 - val_accuracy:

In [ ]:
class_counts = np.bincount(y_test.astype(int))
print(class_counts)


[    0 58951  2493  6725  2923  1277   425]


In [10]:
from sklearn.metrics import confusion_matrix, f1_score

# Predict the test data
y_pred = mlp.predict(X_test1)

# Convert the predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Create the confusion matrix
cm = confusion_matrix(y_test, y_pred_labels)
print("Confusion Matrix:")
print(cm)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred_labels, average='weighted')
print("F1 Score:", f1)




2275/2275 [==============================] - 4s 2ms/step
Confusion Matrix:
[[58477     2    12    93   301    66]
 [    1  2492     0     0     0     0]
 [   12     1  6677     9    22     4]
 [   25     3     0  2853    15    27]
 [   22     0     2     0  1253     0]
 [    1     0     0     0     0   424]]
F1 Score: 0.9918433101708186


In [11]:
mlp.save('./models/mlpmodel.h5')
